Вычитываем текстовые документы и создаем для них TF-IDF матрицу

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from texts_loader import get_texts_from_file
import vectorization
import pandas as pd

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.expand_frame_repr', False)  # Отображение всей таблицы без переноса строк

# Пример данных для классификации
categories = ["технологии", "спорт", "политика"]
labels = [0, 1, 2]

texts = get_texts_from_file()
tfidf_matrix, words, doc_names = vectorization.vectorize(texts)

# Вывод матрицы TF-IDF на экран
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=words, index=doc_names)
print(tfidf_df)

                 nlp    аспект      дело  интересный  ключевой  машинный   область  обучение     очень  полезный   связать     также   учитель       это
Документ 1  0.000000  0.000000  0.399169    0.399169  0.000000  0.235756  0.303578  0.235756  0.399169  0.399169  0.000000  0.000000  0.000000  0.399169
Документ 2  0.483591  0.000000  0.000000    0.000000  0.000000  0.285617  0.367784  0.285617  0.000000  0.000000  0.483591  0.483591  0.000000  0.000000
Документ 3  0.000000  0.459115  0.000000    0.000000  0.459115  0.271161  0.000000  0.542321  0.000000  0.000000  0.000000  0.000000  0.459115  0.000000
